## Import all the important libraries

In [1]:
import os
import pydotplus
import inspect

import pandas as pd
import numpy as np
import xgboost as xgb
import seaborn as sns

from numpy import loadtxt
from scipy import interpolate
from functools import reduce
from scipy.stats import multivariate_normal
from xgboost import XGBClassifier
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier , AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support as score


import matplotlib.pyplot as plt
%matplotlib inline  

from IPython.display import Image

C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Get the current working directory

In [2]:
os.getcwd()

'D:\\INSOFE Internship'

## Read the Sloan Skyserver Digital Survey dataset

In [106]:
data = pd.read_csv("Skyserver.csv",header=0)

## Display the number of observations and number of features in the dataset

In [107]:
data.shape

(10000, 18)

## Display the column names

In [108]:
data.columns

Index(['objid', 'ra', 'dec', 'u', 'g', 'r', 'i', 'z', 'run', 'rerun', 'camcol',
       'field', 'specobjid', 'class', 'redshift', 'plate', 'mjd', 'fiberid'],
      dtype='object')

## Display the index of the dataset

In [109]:
data.index

RangeIndex(start=0, stop=10000, step=1)

## Display the first 3 records to get the feel of the data

In [110]:
data[:3]

,objid,ra,dec,u,g,r,i,z,run,rerun,camcol,field,specobjid,class,redshift,plate,mjd,fiberid
0,1237648704577142822,183.531326,0.089693,19.47406,17.04240,15.94699,15.50342,15.22531,752,301,4,267,3722360139651588096,STAR,-0.000009,3306,54922,491
1,1237648704577142859,183.598371,0.135285,18.66280,17.21449,16.67637,16.48922,16.39150,752,301,4,267,363814405953054720,STAR,-0.000055,323,51615,541
2,1237648704577208477,183.680207,0.126185,19.38298,18.19169,17.47428,17.08732,16.80125,752,301,4,268,323274319570429952,GALAXY,0.123111,287,52023,513


# Display the summary of the data

In [111]:
data.describe(include='all')

,objid,ra,dec,u,g,r,i,z,run,rerun,camcol,field,specobjid,class,redshift,plate,mjd,fiberid
count,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.0,10000.000000,10000.000000,1.000000e+04,10000,10000.000000,10000.000000,10000.000000,10000.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GALAXY,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4998,NaN,NaN,NaN,NaN
mean,1.237650e+18,175.529987,14.836148,18.619355,17.371931,16.840963,16.583579,16.422833,981.034800,301.0,3.648700,302.380100,1.645022e+18,NaN,0.143726,1460.986400,52943.533300,353.069400
std,1.173967e+12,47.783439,25.212207,0.828656,0.945457,1.067764,1.141805,1.203188,273.305024,0.0,1.666183,162.577763,2.013998e+18,NaN,0.388774,1788.778371,1511.150651,206.298149
min,1.237647e+18,8.235100,-5.382632,12.988970,12.799550,12.431600,11.947210,11.610410,308.000000,301.0,1.000000,11.000000,2.995782e+17,NaN,-0.004136,266.000000,51578.000000,1.000000
25%,1.237649e+18,157.370946,-0.539035,18.178035,16.815100,16.173333,15.853705,15.618285,752.000000,301.0,2.000000,184.000000,3.389250e+17,NaN,0.000081,301.000000,51900.000000,186.750000
50%,1.237649e+18,180.394514,0.404166,18.853095,17.495135,16.858770,16.554985,16.389945,756.000000,301.0,4.000000,299.000000,4.966580e+17,NaN,0.042591,441.000000,51997.000000,351.000000
75%,1.237651e+18,201.547279,35.649397,19.259232,18.010145,17.512675,17.258550,17.141447,1331.000000,301.0,5.000000,414.000000,2.881300e+18,NaN,0.092579,2559.000000,54468.000000,510.000000


## Display the data types of the features

In [112]:
data.dtypes

objid          int64
ra           float64
dec          float64
u            float64
g            float64
r            float64
i            float64
z            float64
run            int64
rerun          int64
camcol         int64
field          int64
specobjid     uint64
class         object
redshift     float64
plate          int64
mjd            int64
fiberid        int64
dtype: object

## Rename the old column names with new names for better understanding

In [113]:
#Rename old names with new names
def rename(data, oldnames, newname): 
    if type(oldnames) == str: #input can be a string or list of strings 
        oldnames = [oldnames] #when renaming multiple columns 
        newname = [newname] #make sure you pass the corresponding list of new names
    i = 0 
    for name in oldnames:
        oldvar = oldnames[i]
        data = data.rename(columns = {oldvar : newname[i]})
        i += 1 
    return data   

data=rename(data,['u','g','r','i','z'],['u_band','g_band','r_band','i_band','z_band'])


## Display the data types after giving the new names for the columns

In [114]:
data.dtypes

objid          int64
ra           float64
dec          float64
u_band       float64
g_band       float64
r_band       float64
i_band       float64
z_band       float64
run            int64
rerun          int64
camcol         int64
field          int64
specobjid     uint64
class         object
redshift     float64
plate          int64
mjd            int64
fiberid        int64
dtype: object

## Change data types of columns to float

In [115]:
for col in ['u_band', 'g_band', 'r_band', 'i_band', 'z_band', 
       'field','redshift', 'plate', 'mjd', 'fiberid']:
    data[col] = data[col].astype('float64')

## Unique counts of each columns 

In [116]:
 
col_names=['objid', 'ra', 'dec', 'u_band', 'g_band', 'r_band', 'i_band', 'z_band', 'run', 'rerun', 'camcol',
       'field', 'specobjid', 'class', 'redshift', 'plate', 'mjd', 'fiberid']
unique_values = []
for col in ['objid', 'ra', 'dec', 'u_band', 'g_band', 'r_band', 'i_band', 'z_band', 'run', 'rerun', 'camcol',
       'field', 'specobjid', 'class', 'redshift', 'plate', 'mjd', 'fiberid']:
     unique_values.append(np.size(np.unique(data[col])))

In [117]:
unique_counts = pd.DataFrame({'Feature': col_names,
                              'Unique_Count' : unique_values})    
unique_counts    


,Feature,Unique_Count
0,objid,10000
1,ra,10000
2,dec,10000
3,u_band,9730
4,g_band,9817
5,r_band,9852
6,i_band,9890
7,z_band,9896
8,run,23
9,rerun,1


## Dropping few columns since they are unique and it won't help in buiding the model   

In [118]:
data.drop(['specobjid','rerun','objid'], axis=1, inplace=True)

## Display the count of null values in each column

In [119]:
data.isnull().any()

ra          False
dec         False
u_band      False
g_band      False
r_band      False
i_band      False
z_band      False
run         False
camcol      False
field       False
class       False
redshift    False
plate       False
mjd         False
fiberid     False
dtype: bool

## Rearrange the columns 

In [120]:
data=data[['u_band', 'g_band', 'r_band', 'i_band', 'z_band', 'run', 'camcol',
       'field','redshift', 'plate', 'mjd', 'fiberid','class']]

## Split the categorical and numerical attributes in separate list

In [121]:
print (data.columns, data.shape)

pd.value_counts(data['class'])

cat_Attr = ["camcol","run"]

num_Attr = ['u_band', 'g_band', 'r_band', 'i_band', 'z_band', 
       'field','redshift', 'plate', 'mjd', 'fiberid']

Index(['u_band', 'g_band', 'r_band', 'i_band', 'z_band', 'run', 'camcol',
       'field', 'redshift', 'plate', 'mjd', 'fiberid', 'class'],
      dtype='object') (10000, 13)


## Split the data into Train /Test in 70:30 proportion

In [122]:
X=data[['u_band', 'g_band', 'r_band', 'i_band', 'z_band', 'run', 'camcol',
       'field','redshift', 'plate', 'mjd', 'fiberid']]
y=data[['class']]

In [123]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.30,stratify=y, random_state=42)

## Concat the X_train(independant variables) and y_train(dependant variable)

In [124]:
data_train=pd.concat([X_train,y_train],axis=1)

## Display first three records in train data

In [125]:
data_train.head(3)

,u_band,g_band,r_band,i_band,z_band,run,camcol,field,redshift,plate,mjd,fiberid,class
6013,18.02139,16.15993,15.73891,15.49675,15.42442,1331,3,165.0,0.023890,443.0,51873.0,521.0,GALAXY
1090,19.10976,17.73485,17.16479,16.95688,16.82334,752,4,300.0,-0.000173,2895.0,54567.0,423.0,STAR
8512,19.58240,19.34617,19.26938,19.16666,19.16045,1412,4,211.0,0.722950,496.0,51988.0,549.0,QSO


## Display the number of observations present for each class in target variable for train data

In [126]:
data_train['class'].value_counts()

GALAXY    3499
STAR      2906
QSO        595
Name: class, dtype: int64

## Filter data from train that has only STAR class

In [127]:
data_train_STAR=data_train.loc[data_train['class'] == "STAR"]
data_train_STAR.head(3)

,u_band,g_band,r_band,i_band,z_band,run,camcol,field,redshift,plate,mjd,fiberid,class
1090,19.10976,17.73485,17.16479,16.95688,16.82334,752,4,300.0,-0.000173,2895.0,54567.0,423.0,STAR
1313,18.49873,17.64373,17.32759,17.19038,17.18332,752,2,485.0,0.000081,4028.0,55621.0,291.0,STAR
6424,18.91481,17.96070,17.49830,17.31782,17.22529,1035,3,42.0,-0.000495,6203.0,56266.0,298.0,STAR


## Create KMeans cluster for the STAR class in train data 

In [128]:

A, b = data_train_STAR.loc[:,data_train_STAR.columns!='class'].values, data_train_STAR.loc[:,'class'].values
for k in range (1, 11):
    kmeans_model = KMeans(n_clusters=k, random_state=1).fit(A)
    labels = kmeans_model.labels_
    interia = kmeans_model.inertia_
    print("k:",k, " cost:", interia)

k: 1  cost: 17141126608.778326
k: 2  cost: 7294681537.078241
k: 3  cost: 1440705982.9006073
k: 4  cost: 889498145.9002348
k: 5  cost: 708156687.3212336
k: 6  cost: 609454919.1040789
k: 7  cost: 538025259.7722431
k: 8  cost: 475513898.7634708
k: 9  cost: 419855326.8786144
k: 10  cost: 387099718.2802948


## Based on inertia,select the optimum number of clusters. For STAR class,we created 3 clusters

In [129]:

num_Attr = ['u_band', 'g_band', 'r_band', 'i_band', 'z_band', 
       'field','redshift', 'plate', 'mjd', 'fiberid']
km = KMeans(n_clusters=3, init='k-means++', n_init=10)
km.fit(data_train_STAR[num_Attr])
x = km.fit_predict(data_train_STAR[num_Attr])
x
data_train_STAR["Cluster"]= x
data_train_STAR.head(3)

C:\Users\Ultrabook\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,u_band,g_band,r_band,i_band,z_band,run,camcol,field,redshift,plate,mjd,fiberid,class,Cluster
1090,19.10976,17.73485,17.16479,16.95688,16.82334,752,4,300.0,-0.000173,2895.0,54567.0,423.0,STAR,0
1313,18.49873,17.64373,17.32759,17.19038,17.18332,752,2,485.0,0.000081,4028.0,55621.0,291.0,STAR,0
6424,18.91481,17.96070,17.49830,17.31782,17.22529,1035,3,42.0,-0.000495,6203.0,56266.0,298.0,STAR,1


##  Split the data into different dataframes for each cluster id

In [130]:
data_train_STAR_0=data_train_STAR.loc[data_train_STAR['Cluster'] == 0]
data_train_STAR_1=data_train_STAR.loc[data_train_STAR['Cluster'] == 1]
data_train_STAR_2=data_train_STAR.loc[data_train_STAR['Cluster'] == 2]

## Find the mean and variance for each cluster

In [131]:
mean_STAR_0 = data_train_STAR_0.mean().to_frame()
var_STAR_0 = data_train_STAR_0.var().to_frame()
mean_STAR_1 = data_train_STAR_1.mean().to_frame()
var_STAR_1 = data_train_STAR_1.var().to_frame()
mean_STAR_2 = data_train_STAR_2.mean().to_frame()
var_STAR_2 = data_train_STAR_2.var().to_frame()

## Display the number of data points in each cluster

In [132]:
data_train_STAR['Cluster'].value_counts()

0    1906
2     712
1     288
Name: Cluster, dtype: int64

In [133]:
data_train_GALAXY['Cluster'].value_counts()

0    3412
1      87
Name: Cluster, dtype: int64

In [134]:
data_train_QSO['Cluster'].value_counts()

0    545
1    167
Name: Cluster, dtype: int64

## This function is to build all models for when STAR class is proportioned and calculate the required metrics 

In [140]:
def SelectClusterData(data_train,data_train_STAR,X_test):
    
    prop=[0.1,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5,5.5,6,6.5,7,7.5,8,8.5,9,9.5,10]
    
    X=data_train_STAR[['u_band', 'g_band', 'r_band', 'i_band', 'z_band', 'run', 'camcol',
       'field','redshift', 'plate', 'mjd', 'fiberid','class']]
    y=data_train_STAR[['Cluster']]
    
    X_test=X_test[['u_band', 'g_band', 'r_band', 'i_band', 'z_band', 'run', 'camcol',
       'field','redshift', 'plate', 'mjd', 'fiberid']]
    
      
    data_train_QSO=data_train.loc[data_train['class'] == "QSO"]

    data_train_GALAXY=data_train.loc[data_train['class'] == "GALAXY"]
    
   
    cf_df=pd.DataFrame(columns=['ModelName','Train/Test','Proportion','Confusion matrix','Precision', 'Recall', 
                                'F1Score', 'Support'])
    
    metrics=pd.DataFrame(columns=['ModelName','Train/Test','Proportion','precision_STAR', 'precision_TOT',
                                  'recall_STAR', 'recall_TOT','fscore_STAR','fscore_TOT','fscore_GALAXY','fscore_QSO']) 
    
    model_list = [GaussianNB()
                ,LogisticRegression(random_state=123,solver='newton-cg',verbose=1)
                ,DecisionTreeClassifier()
                ,KNeighborsClassifier()
                ,RandomForestClassifier(n_estimators=7,n_jobs=2)
                ,AdaBoostClassifier()
                ,XGBClassifier()
                ,SVC()]   
    
    
    
    for model_type in model_list:
        
        model_name=str(type(model_type)).split(".")[-1][:-2]
       
        if(model_name.find("Classifier")==-1):
            model_title=model_name
        else:
            model_title=str(type(model_type)).split(".")[-1][:-2][:len("Classifier")]
            
        
            
              
        if (model_name == 'GaussianNB'):
            ModelName='Naive Bayes'
        elif(model_name == 'LogisticRegression'):
            ModelName='Logistic Regression'
        elif(model_name == 'DecisionTreeClassifier'):
            ModelName='Decision Tree'
        elif(model_name == 'SVC'):      
            ModelName='SVC'            
        elif(model_name == 'KNeighborsClassifier'):    
            ModelName='KNN'
        elif(model_name == 'RandomForestClassifier'):     
            ModelName='Random Forest'
        elif(model_name == 'AdaBoostClassifier'): 
            ModelName='AdaBoost'
        elif(model_name == 'XGBClassifier'): 
            ModelName='XGBoost'
               
        metrics_df_train=pd.DataFrame(columns=['ModelName','Train/Test','Proportion','precision_STAR', 'precision_TOT',
                                               'recall_STAR', 'recall_TOT','fscore_STAR','fscore_TOT','fscore_GALAXY',
                                               'fscore_QSO'])
        metrics_df_test=pd.DataFrame(columns=['ModelName','Train/Test','Proportion','precision_STAR', 'precision_TOT',
                                               'recall_STAR', 'recall_TOT','fscore_STAR','fscore_TOT','fscore_GALAXY',
                                               'fscore_QSO'])
                
       
        for item in prop:
            ratio=0.0000

            ratio=round((((7000-2906)*(item/100))/(1-(item/100)))/(2906),4)

            data_STAR, data_r_STAR, label1, label2 = train_test_split( X, y, test_size=ratio,stratify=y,random_state=12)
            
            #print(pd.value_counts(label2['Cluster']))
                       
            data_train=pd.concat([data_train_QSO,data_train_GALAXY,data_r_STAR],axis=0)
                
            X_train=data_train[['u_band', 'g_band', 'r_band', 'i_band', 'z_band', 'run', 'camcol',
                                'field','redshift', 'plate', 'mjd', 'fiberid']]
            y_train=data_train[['class']]   
            
            if (model_name =='SVC' or model_name == 'KNN'):
            #Softnormalize the features for SVM and KNN
                scaler = StandardScaler()
                X_train_std = scaler.fit_transform(X_train)
                X_test_std = scaler.fit_transform(X_test)
               
                clf=model_type
                clf.fit(X_train_std,y_train)
                train_pred=clf.predict(X_train_std)
                test_pred = clf.predict(X_test_std)
            else:
                clf=model_type
                clf.fit(X_train,y_train)
                train_pred=clf.predict(X_train)
                test_pred = clf.predict(X_test)
                
            
            # Train data confusion matrix
            confusion_matrix_train = confusion_matrix(y_train, train_pred)
            precision, recall, fscore, support = score(y_train, train_pred)
           
            fscore=pd.DataFrame(fscore)
  
            cf_df=cf_df.append({'ModelName':ModelName,'Train/Test':'Train','Proportion':item,
                                'Confusion matrix':confusion_matrix_train, 'Precision':precision, 
                                'Recall':recall, "F1Score":fscore, "Support":support}, ignore_index=True)
            fscore=pd.DataFrame(fscore)
            fscore_QSO=round(fscore[0][0],2)
            fscore_GALAXY=round(fscore[0][1],2)
            fscore_STAR=round(fscore[0][2],2) 
            precision=pd.DataFrame(precision)
            precision_STAR=round(precision[0][2],2) 
            recall=pd.DataFrame(recall)
            recall_STAR=round(recall[0][2],2)
            
            
            f1_t=round(f1_score(y_train,train_pred,average="weighted"),2)  
            recall_t=round(recall_score(y_train,train_pred,average="weighted"),2) 
            precision_t=round(precision_score(y_train,train_pred,average="weighted"),2)
            
            
            
   
            metrics_df_train=metrics_df_train.append({'ModelName':ModelName,'Train/Test':'Train','Proportion':item,
                                                      'precision_STAR':precision_STAR,'precision_TOT':precision_t,
                                                      'recall_STAR':recall_STAR,'recall_TOT':recall_t,
                                                      'fscore_STAR':fscore_STAR,'fscore_TOT':f1_t,
                                                      'fscore_GALAXY':fscore_GALAXY,'fscore_QSO':fscore_QSO},ignore_index=True)
               
            # Test data confusion matrix
            confusion_matrix_test= confusion_matrix(y_test, test_pred)
            precision, recall, fscore, support = score(y_test, test_pred) 
           
            
                
            cf_df=cf_df.append({'ModelName':ModelName,'Train/Test':'Test','Proportion':item,
                                'Confusion matrix':confusion_matrix_train, 'Precision':precision, 
                                'Recall':recall, "F1Score":fscore, "Support":support}, ignore_index=True)
                
            fscore=pd.DataFrame(fscore)
            fscore_QSO=round(fscore[0][0],2)
            fscore_GALAXY=round(fscore[0][1],2)
            fscore_STAR=round(fscore[0][2],2) 
            precision=pd.DataFrame(precision)
            precision_STAR=round(precision[0][2],2) 
            recall=pd.DataFrame(recall)
            recall_STAR=round(recall[0][2],2)
            
            f1_t=round(f1_score(y_test,test_pred,average="weighted"),2)
            recall_t=round(recall_score(y_test,test_pred,average="weighted"),2) 
            precision_t=round(precision_score(y_test,test_pred,average="weighted"),2) 
            metrics_df_test=metrics_df_test.append({'ModelName':ModelName,'Train/Test':'Test','Proportion':item,
                                                    'precision_STAR':precision_STAR,'precision_TOT':precision_t,
                                                    'recall_STAR':recall_STAR,'recall_TOT':recall_t,
                                                    'fscore_STAR':fscore_STAR,'fscore_TOT':f1_t,
                                                    'fscore_GALAXY':fscore_GALAXY,'fscore_QSO':fscore_QSO},ignore_index=True)
            
            metrics_df=pd.concat([metrics_df_train,metrics_df_test],axis=0)
           
        metrics=pd.concat([metrics,metrics_df],axis=0)
    return metrics

## Metrics for STAR class is stored in Metrics_STAR dataframe

In [141]:
Metrics_STAR = SelectClusterData(data_train,data_train_STAR,X_test)

C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to

C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\optimize.

C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    2.0s finished
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Ultrabook\Anaconda3\li

C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.9s finished
C:\Users\Ultrabook\Anaconda3\li

C:\Users\Ultrabook\Anaconda3\lib\site-packages\ipykernel_launcher.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
C:\Users\Ultrabook\Anaconda3\lib\site-packages\ipykernel_launcher.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
C:\Users\Ultrabook\Anaconda3\lib\site-packages\ipykernel_launcher.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
C:\Users\Ultrabook\Anaconda3\lib\site-packages\ipykernel_launcher.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
C:\Users\Ultrabook\Anaconda3\lib\site-packages\ipykernel_launcher.py:99: DataConvers

C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to

C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in f

C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Pl

C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in f

C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [146]:
Metrics_STAR[126:147]

,ModelName,Train/Test,Proportion,precision_STAR,precision_TOT,recall_STAR,recall_TOT,fscore_STAR,fscore_TOT,fscore_GALAXY,fscore_QSO
0,KNN,Train,0.1,0.00,0.86,0.00,0.88,0.00,0.85,0.38,0.93
1,KNN,Train,0.5,0.83,0.86,0.24,0.88,0.37,0.85,0.37,0.93
2,KNN,Train,1.0,0.71,0.86,0.40,0.87,0.52,0.84,0.37,0.93
3,KNN,Train,1.5,0.74,0.86,0.49,0.87,0.59,0.84,0.36,0.93
4,KNN,Train,2.0,0.76,0.85,0.52,0.87,0.62,0.84,0.36,0.93
5,KNN,Train,2.5,0.81,0.85,0.55,0.87,0.66,0.84,0.36,0.93
6,KNN,Train,3.0,0.82,0.85,0.58,0.87,0.68,0.84,0.36,0.93
7,KNN,Train,3.5,0.85,0.85,0.60,0.87,0.71,0.84,0.36,0.93
8,KNN,Train,4.0,0.87,0.85,0.61,0.87,0.72,0.84,0.36,0.93
9,KNN,Train,4.5,0.86,0.85,0.62,0.86,0.72,0.84,0.36,0.92


## Metrics for STAR class is written to an output file

In [32]:
Metrics_STAR.to_csv('MetricsPropSTAR.csv', index = False)

## Filter data from train that has only GALAXY class

In [33]:
data_train_GALAXY=data_train.loc[data_train['class'] == "GALAXY"]

## Create KMeans cluster for the GALAXY class in train data 

In [34]:

C, d = data_train_GALAXY.loc[:,data_train_GALAXY.columns!='class'].values, data_train_GALAXY.loc[:,'class'].values
for k in range (1, 11):
    kmeans_model = KMeans(n_clusters=k, random_state=1).fit(C)
    labels = kmeans_model.labels_
    interia = kmeans_model.inertia_
    print("k:",k, " cost:", interia)

k: 1  cost: 3365527068.069581
k: 2  cost: 967876378.0778059
k: 3  cost: 665282902.7594136
k: 4  cost: 440854939.3402508
k: 5  cost: 366065205.66297275
k: 6  cost: 313868379.13662845
k: 7  cost: 272069103.68871933
k: 8  cost: 244852337.3205732
k: 9  cost: 215915564.15024877
k: 10  cost: 193436036.81091982


## Based on inertia,select the optimum number of clusters. For GALAXY class,we created  2 clusters

In [35]:

num_Attr = ['u_band', 'g_band', 'r_band', 'i_band', 'z_band', 
       'field','redshift', 'plate', 'mjd', 'fiberid']
km = KMeans(n_clusters=2, init='k-means++', n_init=10)
km.fit(data_train_GALAXY[num_Attr])
c = km.fit_predict(data_train_GALAXY[num_Attr])
c
data_train_GALAXY["Cluster"]= c
data_train_GALAXY.head(3)

C:\Users\Ultrabook\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,u_band,g_band,r_band,i_band,z_band,run,camcol,field,redshift,plate,mjd,fiberid,class,Cluster
6013,18.02139,16.15993,15.73891,15.49675,15.42442,1331,3,165.0,0.023890,443.0,51873.0,521.0,GALAXY,0
8929,19.41348,18.06345,17.32228,16.94317,16.67798,1350,3,102.0,0.140871,436.0,51883.0,511.0,GALAXY,0
3976,18.87733,17.88631,17.48114,17.25515,17.13066,1140,1,164.0,0.026482,330.0,52370.0,294.0,GALAXY,0


##  Split the data into different dataframes for each cluster id

In [36]:
data_train_GALAXY_0=data_train_GALAXY.loc[data_train_GALAXY['Cluster'] == 0]
data_train_GALAXY_1=data_train_GALAXY.loc[data_train_GALAXY['Cluster'] == 1]

## Calculate the mean and variance of each cluster

In [37]:
mean_GALAXY_0 = data_train_GALAXY_0.mean().to_frame()
var_GALAXY_0 = data_train_GALAXY_0.var().to_frame()
mean_GALAXY_1 = data_train_GALAXY_1.mean().to_frame()
var_GALAXY_1 = data_train_GALAXY_1.var().to_frame()

## Display the number of data points in each cluster

In [38]:
data_train_GALAXY['Cluster'].value_counts()

0    3412
1      87
Name: Cluster, dtype: int64

## This function is to build all models for when GALAXY class is  proportioned and calculate the required metrics 

In [91]:
def SelectClusterDataG(data_train,data_train_GALAXY,X_test):
    
    prop=[0.1,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5,5.5,6,6.5,7,7.5,8,8.5,9,9.5,10]
    
    X=data_train_GALAXY[['u_band', 'g_band', 'r_band', 'i_band', 'z_band', 'run', 'camcol',
       'field','redshift', 'plate', 'mjd', 'fiberid','class']]
    y=data_train_GALAXY[['Cluster']]
    
    X_test=X_test[['u_band', 'g_band', 'r_band', 'i_band', 'z_band', 'run', 'camcol',
       'field','redshift', 'plate', 'mjd', 'fiberid']]
    
      
    data_train_QSO=data_train.loc[data_train['class'] == "QSO"]

    data_train_STAR=data_train.loc[data_train['class'] == "STAR"]
    
   
    cf_df=pd.DataFrame(columns=['ModelName','Train/Test','Proportion','Confusion matrix','Precision', 'Recall', 
                                'F1Score', 'Support'])
    
    metrics=pd.DataFrame(columns=['ModelName','Train/Test','Proportion','precision_GALAXY', 'precision_TOT',
                                  'recall_GALAXY', 'recall_TOT','fscore_GALAXY','fscore_TOT','fscore_STAR','fscore_QSO']) 
    
    model_list = [GaussianNB()
                ,LogisticRegression(random_state=123,solver='newton-cg',verbose=1)
                ,DecisionTreeClassifier()
                ,KNeighborsClassifier()
                ,RandomForestClassifier(n_estimators=7,n_jobs=2)
                ,AdaBoostClassifier()
                ,XGBClassifier()
                ,SVC()]   
    
    

    for model_type in model_list:
        
        model_name=str(type(model_type)).split(".")[-1][:-2]
        if(model_name.find("Classifier")==-1):
            model_title=model_name
        else:
            model_title=str(type(model_type)).split(".")[-1][:-2][:len("Classifier")]
            
        
        
        if (model_name == 'GaussianNB'):
            ModelName='Naive Bayes'
        elif(model_name == 'LogisticRegression'):
            ModelName='Logistic Regression'
        elif(model_name == 'DecisionTreeClassifier'):
            ModelName='Decision Tree'
        elif(model_name == 'SVC'):      
            ModelName='SVC'            
        elif(model_name == 'KNeighborsClassifier'):    
            ModelName='KNN'
        elif(model_name == 'RandomForestClassifier'):     
            ModelName='Random Forest'
        elif(model_name == 'AdaBoostClassifier'): 
            ModelName='AdaBoost'
        elif(model_name == 'XGBClassifier'): 
            ModelName='XGBoost'
               
        metrics_df_train=pd.DataFrame(columns=['ModelName','Train/Test','Proportion','precision_GALAXY', 'precision_TOT',
                                               'recall_GALAXY', 'recall_TOT','fscore_GALAXY','fscore_TOT','fscore_STAR',
                                               'fscore_QSO'])
        metrics_df_test=pd.DataFrame(columns=['ModelName','Train/Test','Proportion','precision_GALAXY', 'precision_TOT',
                                              'recall_GALAXY', 'recall_TOT','fscore_GALAXY','fscore_TOT','fscore_STAR',
                                              'fscore_QSO'])
                
       
        for item in prop:
            ratio=0.0000

            ratio=round((((7000-3499)*(item/100))/(1-(item/100)))/(3499),4)

            data_GALAXY, data_r_GALAXY, label1, label2 = train_test_split( X, y, test_size=ratio,stratify=y,random_state=12)
                                     
            data_train=pd.concat([data_train_QSO,data_train_STAR,data_r_GALAXY],axis=0)
                
            X_train=data_train[['u_band', 'g_band', 'r_band', 'i_band', 'z_band', 'run', 'camcol',
                                'field','redshift', 'plate', 'mjd', 'fiberid']]
            y_train=data_train[['class']]
            
            print(pd.value_counts(label2['Cluster']))
            
            if (model_name =='SVC' or model_name == 'KNN'):
            #Softnormalize the features for SVM and KNN
                scaler = StandardScaler()
                X_train_std = scaler.fit_transform(X_train)
                X_test_std = scaler.fit_transform(X_test)
               
                clf=model_type
                clf.fit(X_train_std,y_train)
                train_pred=clf.predict(X_train_std)
                test_pred = clf.predict(X_test_std)
            else:
                clf=model_type
                clf.fit(X_train,y_train)
                train_pred=clf.predict(X_train)
                test_pred = clf.predict(X_test)
                
            
            # Train data confusion matrix
            confusion_matrix_train = confusion_matrix(y_train, train_pred)
            precision, recall, fscore, support = score(y_train, train_pred)
           
            fscore=pd.DataFrame(fscore)
  
            cf_df=cf_df.append({'ModelName':ModelName,'Train/Test':'Train','Proportion':item,
                                'Confusion matrix':confusion_matrix_train, 'Precision':precision, 
                                'Recall':recall, "F1Score":fscore, "Support":support}, ignore_index=True)
            fscore=pd.DataFrame(fscore)
            fscore_QSO=round(fscore[0][0],2)
            fscore_STAR=round(fscore[0][1],2)
            fscore_GALAXY=round(fscore[0][2],2) 
            precision=pd.DataFrame(precision)
            precision_GALAXY=round(precision[0][2],2) 
            recall=pd.DataFrame(recall)
            recall_GALAXY=round(recall[0][2],2)
            
            
            f1_t=round(f1_score(y_train,train_pred,average="weighted"),2)  
            recall_t=round(recall_score(y_train,train_pred,average="weighted"),2) 
            precision_t=round(precision_score(y_train,train_pred,average="weighted"),2)
            
            
            
   
            metrics_df_train=metrics_df_train.append({'ModelName':ModelName,'Train/Test':'Train','Proportion':item,
                                                      'precision_GALAXY':precision_GALAXY,'precision_TOT':precision_t,
                                                      'recall_GALAXY':recall_GALAXY,'recall_TOT':recall_t,
                                                      'fscore_GALAXY':fscore_GALAXY,'fscore_TOT':f1_t,
                                                      'fscore_STAR':fscore_STAR,'fscore_QSO':fscore_QSO},ignore_index=True)
               
            # Test data confusion matrix
            confusion_matrix_test= confusion_matrix(y_test, test_pred)
            precision, recall, fscore, support = score(y_test, test_pred) 
           
            
                
            cf_df=cf_df.append({'ModelName':ModelName,'Train/Test':'Test','Proportion':item,
                                'Confusion matrix':confusion_matrix_train, 'Precision':precision, 
                                'Recall':recall, "F1Score":fscore, "Support":support}, ignore_index=True)
                
            fscore=pd.DataFrame(fscore)
            fscore_QSO=round(fscore[0][0],2)
            fscore_STAR=round(fscore[0][1],2)
            fscore_GALAXY=round(fscore[0][2],2) 
            precision=pd.DataFrame(precision)
            precision_GALAXY=round(precision[0][2],2) 
            recall=pd.DataFrame(recall)
            recall_GALAXY=round(recall[0][2],2)
            
            f1_t=round(f1_score(y_test,test_pred,average="weighted"),2)
            recall_t=round(recall_score(y_test,test_pred,average="weighted"),2) 
            precision_t=round(precision_score(y_test,test_pred,average="weighted"),2) 
            metrics_df_test=metrics_df_test.append({'ModelName':ModelName,'Train/Test':'Test','Proportion':item,
                                                    'precision_GALAXY':precision_GALAXY,'precision_TOT':precision_t,
                                                    'recall_GALAXY':recall_GALAXY,'recall_TOT':recall_t,
                                                    'fscore_GALAXY':fscore_GALAXY,'fscore_TOT':f1_t,
                                                    'fscore_STAR':fscore_STAR,'fscore_QSO':fscore_QSO},ignore_index=True)
            
            metrics_df=pd.concat([metrics_df_train,metrics_df_test],axis=0)
           
        metrics=pd.concat([metrics,metrics_df],axis=0)
    return metrics

## Metrics for GALAXY class is stored in Metrics_GALAXY dataframe

In [92]:
Metrics_GALAXY = SelectClusterDataG(data_train,data_train_GALAXY,X_test)

0    4
Name: Cluster, dtype: int64


C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0    18
Name: Cluster, dtype: int64


C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0    35
1     1
Name: Cluster, dtype: int64
0    53
1     1
Name: Cluster, dtype: int64


...

C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0    359
1      9
Name: Cluster, dtype: int64


C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0    380
1     10
Name: Cluster, dtype: int64


C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## Metrics of GALAXY is written to output file

In [41]:
Metrics_GALAXY.to_csv('MetricsPropGal.csv', index = False)

## Filter data from train that has only QSO class

In [42]:
data_train_QSO=data_train.loc[data_train['class'] == "QSO"]
data_train_QSO.head(3)

,u_band,g_band,r_band,i_band,z_band,run,camcol,field,redshift,plate,mjd,fiberid,class
8512,19.58240,19.34617,19.26938,19.16666,19.16045,1412,4,211.0,0.722950,496.0,51988.0,549.0,QSO
1055,19.30088,19.17728,19.26831,19.40944,19.54679,756,3,272.0,1.350456,3832.0,55289.0,386.0,QSO
4070,18.18040,17.81555,17.49082,17.44982,17.41562,756,3,343.0,1.170512,278.0,51900.0,232.0,QSO


## Create KMeans cluster for the QSO class in train data

In [43]:

E, f = data_train_QSO.loc[:,data_train_QSO.columns!='class'].values, data_train_QSO.loc[:,'class'].values
for k in range (1, 11):
    kmeans_model = KMeans(n_clusters=k, random_state=1).fit(E)
    labels = kmeans_model.labels_
    interia = kmeans_model.inertia_
    print("k:",k, " cost:", interia)

k: 1  cost: 3854219010.868726
k: 2  cost: 470650821.6714021
k: 3  cost: 145138888.66443112
k: 4  cost: 100337830.70937124
k: 5  cost: 84995955.26046583
k: 6  cost: 72976759.22608489
k: 7  cost: 66065954.47400062
k: 8  cost: 59663606.304597795
k: 9  cost: 55351182.73964767
k: 10  cost: 49866472.00533344


## Based on inertia,select the optimum number of clusters. For GALAXY class,we created  2 clusters

In [44]:

num_Attr = ['u_band', 'g_band', 'r_band', 'i_band', 'z_band', 
       'field','redshift', 'plate', 'mjd', 'fiberid']
km = KMeans(n_clusters=2, init='k-means++', n_init=10)
km.fit(data_train_QSO[num_Attr])
c = km.fit_predict(data_train_QSO[num_Attr])
c
data_train_QSO["Cluster"]= c
data_train_QSO.head(3)

C:\Users\Ultrabook\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,u_band,g_band,r_band,i_band,z_band,run,camcol,field,redshift,plate,mjd,fiberid,class,Cluster
8512,19.58240,19.34617,19.26938,19.16666,19.16045,1412,4,211.0,0.722950,496.0,51988.0,549.0,QSO,0
1055,19.30088,19.17728,19.26831,19.40944,19.54679,756,3,272.0,1.350456,3832.0,55289.0,386.0,QSO,1
4070,18.18040,17.81555,17.49082,17.44982,17.41562,756,3,343.0,1.170512,278.0,51900.0,232.0,QSO,0


## Display the number of data points in each cluster

In [45]:
data_train_QSO['Cluster'].value_counts()

0    486
1    109
Name: Cluster, dtype: int64

##  Split the data into different dataframes for each cluster id

In [46]:
data_train_QSO_0=data_train_QSO.loc[data_train_QSO['Cluster'] == 0]
data_train_QSO_1=data_train_QSO.loc[data_train_QSO['Cluster'] == 1]

## Calculate the mean and variance of each cluster 

In [47]:
mean_QSO_0 = data_train_QSO_0.mean()
var_QSO_0 = data_train_QSO_0.var()
mean_QSO_1 = data_train_QSO_1.mean()
var_QSO_1 = data_train_QSO_1.var()

## Calculate the covariance for the clustered data and find the diagonal of the QSO data

In [48]:
cov_QSO_0 = data_train_QSO_0.cov()
cov_QSO_0d = np.diag(var_QSO_0)

In [49]:
cov_QSO_1 = data_train_QSO_1.cov()
cov_QSO_1d = np.diag(var_QSO_1)

## Generate random samples from multi-variate normal distribution using calculated mean and vaiance of each cluster

In [50]:
mn_0 = np.random.multivariate_normal(mean=mean_QSO_0, cov=cov_QSO_0d, size=59)
mn_1 = np.random.multivariate_normal(mean=mean_QSO_1, cov=cov_QSO_1d, size=58)

## Convert the generated random data points to dataframe

In [51]:
mn_0 = pd.DataFrame(mn_0)
mn_1 = pd.DataFrame(mn_1)

## Combine the generated random data samples of each cluster into a single dataframe

In [52]:
mn = mn_0.append(mn_1, ignore_index=True)

## Define column names for the generated data samples as we need to append this generated data samples with the existiong data of QSO class

In [53]:
mn.columns = ['u_band','g_band','r_band','i_band','z_band','run','camcol','field','redshift','plate','mjd','fiberid','Cluster']

In [54]:
mn['class'] = "QSO"

## Display the generated data points

In [55]:
mn.head(3)

,u_band,g_band,r_band,i_band,z_band,run,camcol,field,redshift,plate,mjd,fiberid,Cluster,class
0,17.951538,18.634097,17.609232,18.540853,17.411591,1151.606671,2.807005,249.466478,1.033866,215.610787,51954.050873,383.11097,0.0,QSO
1,19.827937,17.456946,18.471665,19.882003,19.295587,1296.296817,5.046051,495.115992,1.814605,127.668531,52204.003594,254.26177,0.0,QSO
2,19.263590,18.781654,18.143744,18.690722,16.799366,644.072428,-0.044718,234.344990,2.228791,125.320253,52073.840135,385.87385,0.0,QSO


## Convert the cluster id to integer

In [56]:
mn.loc[:, 'Cluster'] = mn['Cluster'].astype(int)

## Reorder columns

In [57]:
cols = mn.columns.tolist()
# reorder columns
cols = cols[0:12] + [cols[13]] + [cols[12]]
# "commit" the reordering
mn = mn[cols]
mn.head(2)

,u_band,g_band,r_band,i_band,z_band,run,camcol,field,redshift,plate,mjd,fiberid,class,Cluster
0,17.951538,18.634097,17.609232,18.540853,17.411591,1151.606671,2.807005,249.466478,1.033866,215.610787,51954.050873,383.11097,QSO,0
1,19.827937,17.456946,18.471665,19.882003,19.295587,1296.296817,5.046051,495.115992,1.814605,127.668531,52204.003594,254.26177,QSO,0


## Append the generated data points to the existing QSO data

In [58]:
data_train_QSO = data_train_QSO.append(mn, ignore_index=True)

## Display the number of data points in each cluster after cluster based sampling

In [59]:
data_train_QSO['Cluster'].value_counts()

0    545
1    167
Name: Cluster, dtype: int64

## Find the mean and variance of each cluster after generating cluster based sampling

In [60]:
mean_QSO_0 = data_train_QSO_0.mean().to_frame()
var_QSO_0 = data_train_QSO_0.var().to_frame()
mean_QSO_1 = data_train_QSO_1.mean().to_frame()
var_QSO_1 = data_train_QSO_1.var().to_frame()

## This function is to build all models for when QSO class is  proportioned and calculate the required metrics 

In [94]:
def SelectClusterDataQ(data_train,data_train_QSO,X_test):
    
    prop=[0.1,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5,5.5,6,6.5,7,7.5,8,8.5,9,9.5,10]
    
    X=data_train_QSO[['u_band', 'g_band', 'r_band', 'i_band', 'z_band', 'run', 'camcol',
       'field','redshift', 'plate', 'mjd', 'fiberid','class']]
    y=data_train_QSO[['Cluster']]
    
    X_test=X_test[['u_band', 'g_band', 'r_band', 'i_band', 'z_band', 'run', 'camcol',
       'field','redshift', 'plate', 'mjd', 'fiberid']]
    
      
    data_train_GALAXY=data_train.loc[data_train['class'] == "GALAXY"]

    data_train_STAR=data_train.loc[data_train['class'] == "STAR"]
    
   
    cf_df=pd.DataFrame(columns=['ModelName','Train/Test','Proportion','Confusion matrix','Precision', 'Recall', 
                                'F1Score', 'Support'])
    
    metrics=pd.DataFrame(columns=['ModelName','Train/Test','Proportion','precision_QSO', 'precision_TOT',
                                  'recall_QSO', 'recall_TOT','fscore_QSO','fscore_TOT','fscore_GALAXY','fscore_STAR']) 
    
    model_list = [GaussianNB()
                ,LogisticRegression(random_state=123,solver='newton-cg',verbose=1)
                ,DecisionTreeClassifier()
                ,KNeighborsClassifier()
                ,RandomForestClassifier(n_estimators=7,n_jobs=2)
                ,AdaBoostClassifier()
                ,XGBClassifier()
                ,SVC()]   
    
    
    for model_type in model_list:
        
        model_name=str(type(model_type)).split(".")[-1][:-2]
        if(model_name.find("Classifier")==-1):
            model_title=model_name
        else:
            model_title=str(type(model_type)).split(".")[-1][:-2][:len("Classifier")]
            
        
        if (model_name == 'GaussianNB'):
            ModelName='Naive Bayes'
        elif(model_name == 'LogisticRegression'):
            ModelName='Logistic Regression'
        elif(model_name == 'DecisionTreeClassifier'):
            ModelName='Decision Tree'
        elif(model_name == 'SVC'):      
            ModelName='SVC'            
        elif(model_name == 'KNeighborsClassifier'):    
            ModelName='KNN'
        elif(model_name == 'RandomForestClassifier'):     
            ModelName='Random Forest'
        elif(model_name == 'AdaBoostClassifier'): 
            ModelName='AdaBoost'
        elif(model_name == 'XGBClassifier'): 
            ModelName='XGBoost'
               
        metrics_df_train=pd.DataFrame(columns=['ModelName','Train/Test','Proportion','precision_QSO', 'precision_TOT',
                                               'recall_QSO', 'recall_TOT','fscore_QSO','fscore_TOT','fscore_GALAXY',
                                               'fscore_STAR'])
        metrics_df_test=pd.DataFrame(columns=['ModelName','Train/Test','Proportion','precision_QSO', 'precision_TOT',
                                              'recall_QSO', 'recall_TOT','fscore_QSO','fscore_TOT','fscore_GALAXY',
                                              'fscore_STAR'])
                
       
        for item in prop:
            ratio=0.0000

            ratio=round((((7000-712)*(item/100))/(1-(item/100)))/(712),4)

            data_QSO, data_r_QSO, label1, label2 = train_test_split( X, y, test_size=ratio,stratify=y,random_state=12)
                                      
            data_train=pd.concat([data_train_GALAXY,data_train_STAR,data_r_QSO],axis=0)
                
            X_train=data_train[['u_band', 'g_band', 'r_band', 'i_band', 'z_band', 'run', 'camcol',
                                'field','redshift', 'plate', 'mjd', 'fiberid']]
            y_train=data_train[['class']]   
            
            if (model_name =='SVC' or model_name == 'KNN'):
            #Softnormalize the features for SVM and KNN
                scaler = StandardScaler()
                X_train_std = scaler.fit_transform(X_train)
                X_test_std = scaler.fit_transform(X_test)
               
                clf=model_type
                clf.fit(X_train_std,y_train)
                train_pred=clf.predict(X_train_std)
                test_pred = clf.predict(X_test_std)
            else:
                clf=model_type
                clf.fit(X_train,y_train)
                train_pred=clf.predict(X_train)
                test_pred = clf.predict(X_test)
                
            
            # Train data confusion matrix
            confusion_matrix_train = confusion_matrix(y_train, train_pred)
            precision, recall, fscore, support = score(y_train, train_pred)
           
            fscore=pd.DataFrame(fscore)
  
            cf_df=cf_df.append({'ModelName':ModelName,'Train/Test':'Train','Proportion':item,
                                'Confusion matrix':confusion_matrix_train, 'Precision':precision, 
                                'Recall':recall, "F1Score":fscore, "Support":support}, ignore_index=True)
            fscore=pd.DataFrame(fscore)
            fscore_GALAXY=round(fscore[0][0],2)
            fscore_STAR=round(fscore[0][1],2)
            fscore_QSO=round(fscore[0][2],2) 
            precision=pd.DataFrame(precision)
            precision_QSO=round(precision[0][2],2) 
            recall=pd.DataFrame(recall)
            recall_QSO=round(recall[0][2],2)
            
            
            f1_t=round(f1_score(y_train,train_pred,average="weighted"),2)  
            recall_t=round(recall_score(y_train,train_pred,average="weighted"),2) 
            precision_t=round(precision_score(y_train,train_pred,average="weighted"),2)
            
            
            
   
            metrics_df_train=metrics_df_train.append({'ModelName':ModelName,'Train/Test':'Train','Proportion':item,
                                                      'precision_QSO':precision_QSO,'precision_TOT':precision_t,
                                                      'recall_QSO':recall_QSO,'recall_TOT':recall_t,
                                                      'fscore_QSO':fscore_QSO,'fscore_TOT':f1_t,
                                                      'fscore_GALAXY':fscore_GALAXY,'fscore_STAR':fscore_STAR},ignore_index=True)
               
            # Test data confusion matrix
            confusion_matrix_test= confusion_matrix(y_test, test_pred)
            precision, recall, fscore, support = score(y_test, test_pred) 
           
            
                
            cf_df=cf_df.append({'ModelName':ModelName,'Train/Test':'Test','Proportion':item,
                                'Confusion matrix':confusion_matrix_train, 'Precision':precision, 
                                'Recall':recall, "F1Score":fscore, "Support":support}, ignore_index=True)
                
            fscore=pd.DataFrame(fscore)
            fscore_GALAXY=round(fscore[0][0],2)
            fscore_STAR=round(fscore[0][1],2)
            fscore_QSO=round(fscore[0][2],2) 
            precision=pd.DataFrame(precision)
            precision_QSO=round(precision[0][2],2) 
            recall=pd.DataFrame(recall)
            recall_QSO=round(recall[0][2],2)
            
            f1_t=round(f1_score(y_test,test_pred,average="weighted"),2)
            recall_t=round(recall_score(y_test,test_pred,average="weighted"),2) 
            precision_t=round(precision_score(y_test,test_pred,average="weighted"),2) 
            metrics_df_test=metrics_df_test.append({'ModelName':ModelName,'Train/Test':'Test','Proportion':item,
                                                    'precision_QSO':precision_QSO,'precision_TOT':precision_t,
                                                    'recall_QSO':recall_QSO,'recall_TOT':recall_t,
                                                    'fscore_QSO':fscore_QSO,'fscore_TOT':f1_t,
                                                    'fscore_GALAXY':fscore_GALAXY,'fscore_STAR':fscore_STAR},ignore_index=True)
            
            metrics_df=pd.concat([metrics_df_train,metrics_df_test],axis=0)
           
        metrics=pd.concat([metrics,metrics_df],axis=0)
    return metrics

## Metrics for QSO class is stored in Metrics_QSO dataframe

In [62]:
Metrics_QSO = SelectClusterDataQ(data_train,data_train_QSO,X_test)

C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to

C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    2.1s finished
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
C:\Users\Ultrabook\Anaconda3\li

C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    2.1s finished
C:\Users\Ultrabook\Anaconda3\li

C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    2.3s finished
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
C:\Users\Ultrabook\Anaconda3\li

C:\Users\Ultrabook\Anaconda3\lib\site-packages\ipykernel_launcher.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
C:\Users\Ultrabook\Anaconda3\lib\site-packages\ipykernel_launcher.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
C:\Users\Ultrabook\Anaconda3\lib\site-packages\ipykernel_launcher.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
C:\Users\Ultrabook\Anaconda3\lib\site-packages\ipykernel_launcher.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
C:\Users\Ultrabook\Anaconda3\lib\site-packages\ipykernel_launcher.py:93: DataConvers

C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to

C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Pl

C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in f

C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Ultrabook\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Pl

## Metrics of QSO class is written to output file

In [63]:
Metrics_QSO.to_csv('MetricsPropQso.csv', index = False)

## Combine Metrics of STAR , GALAXY and QSO into single dataframe

In [64]:
frames = [Metrics_STAR, Metrics_GALAXY,Metrics_QSO]
result = pd.concat(frames)

C:\Users\Ultrabook\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  


## Write the combined Metrics to an output file

In [65]:
result.to_csv('results.csv', index = False)

## Mean and Variance of each cluster of all the classes are combined into a single dataframe

In [66]:
mean_var_df=pd.concat([mean_GALAXY_0,var_GALAXY_0,mean_GALAXY_1,var_GALAXY_1,mean_STAR_0,var_STAR_0,mean_STAR_1,var_STAR_1,mean_STAR_2,var_STAR_2,mean_QSO_0,var_QSO_0,mean_QSO_1,var_QSO_1],axis=1)

mean_var_df.columns=[['mean_GALAXY_0','var_GALAXY_0','mean_GALAXY_1','var_GALAXY_1','mean_STAR_0','var_STAR_0','mean_STAR_1','var_STAR_1','mean_STAR_2','var_STAR_2','mean_QSO_0','var_QSO_0','mean_QSO_1','var_QSO_1']]


## Display records of combined mean and variance

In [67]:
mean_var_df

,mean_GALAXY_0,var_GALAXY_0,mean_GALAXY_1,var_GALAXY_1,mean_STAR_0,var_STAR_0,mean_STAR_1,var_STAR_1,mean_STAR_2,var_STAR_2,mean_QSO_0,var_QSO_0,mean_QSO_1,var_QSO_1
u_band,18.802331,0.485123,18.607606,7.136678e-01,18.274333,9.147086e-01,1.862354e+01,6.423274e-01,18.432934,7.686203e-01,18.933092,0.344866,18.896709,4.300215e-01
g_band,17.349589,0.588738,17.034324,7.687653e-01,17.024054,1.027956e+00,1.751412e+01,6.274696e-01,17.338475,8.410515e-01,18.681489,0.416914,18.578456,4.062017e-01
r_band,16.649565,0.740433,16.298610,1.004562e+00,16.583599,1.197937e+00,1.714848e+01,7.361827e-01,17.041311,1.052544e+00,18.489406,0.503790,18.467023,4.404075e-01
i_band,16.271668,0.807659,15.936748,1.091882e+00,16.426258,1.374262e+00,1.702431e+01,8.263272e-01,16.962605,1.214923e+00,18.343810,0.583661,18.363661,4.584698e-01
z_band,16.014759,0.912115,15.707202,1.348204e+00,16.351384,1.382137e+00,1.698723e+01,9.201931e-01,16.903865,1.230767e+00,18.263422,0.672636,18.281207,5.417899e-01
run,992.734760,73438.540298,1156.390805,8.632399e+04,920.285414,6.563809e+04,1.170847e+03,6.901324e+04,935.577247,6.436911e+04,1022.886831,77328.929434,1080.990826,8.044410e+04
camcol,3.650645,2.865309,3.747126,2.098102e+00,3.454879,2.677491e+00,4.371528e+00,1.655911e+00,3.810393,2.817726e+00,3.699588,2.882763,3.706422,2.561162e+00
field,300.708382,27341.720563,302.034483,2.771020e+04,301.229276,1.950371e+04,1.967222e+02,2.149486e+04,344.452247,3.321119e+04,304.786008,25972.494319,316.458716,2.492229e+04
redshift,0.079695,0.001882,0.100501,1.930839e-02,0.000068,1.447490e-07,2.345464e-07,1.501506e-07,0.000040,2.282940e-07,1.116713,0.370942,1.697587,7.829190e-01
plate,368.441383,12620.288266,4370.114943,2.504198e+06,2901.734523,2.465083e+05,7.247326e+03,2.287792e+05,370.657303,1.806266e+04,389.736626,24301.765542,5159.972477,2.664642e+06


## Write to mean and variance of each cluster to output file

In [ ]:
mean_var_df.to_csv('mean_var_df.csv', index = True)

## Store Model names in a list

In [68]:
model_names= ['Naive Bayes'
             ,'Logistic Regression'
             ,'Decision Tree'
             ,'KNN'
             ,'Random Forest'
             ,'AdaBoost'
             ,'XGBoost'
             ,'SVC']   

## Calculate New metric score for the STAR class and total score when STAR class is proportioned

In [69]:
STAR_train=Metrics_STAR[Metrics_STAR['Train/Test']=='Train']
STAR_test=Metrics_STAR[Metrics_STAR['Train/Test']=='Test']

scores_STAR=pd.DataFrame(columns=['ModelName','Proportion','fscore_STAR_train','fscore_TOT_train','fscore_STAR_test',
                                  'fscore_TOT_test','S_STAR','S_STAR_TOT'])

for model in model_names:
    prop=[0.1,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5,5.5,6,6.5,7,7.5,8,8.5,9,9.5,10]

    for i in prop:
        for index, row in STAR_train.iterrows():
        
            if (row['ModelName'] == model and row['Proportion']==i):
                A=row['fscore_STAR']
                C=row['fscore_TOT']
                
        for index, row in STAR_test.iterrows():
            if (row['ModelName'] == model and row['Proportion']==i):
                B=row['fscore_STAR']
                D=row['fscore_TOT']
            
        S_STAR=round((A+B)/(2*(1+abs(A-B))),2)
        S_STAR_TOT=round((C+D)/(2*(1+abs(C-D))),2)
        scores_STAR=scores_STAR.append({'ModelName':model,'Proportion':i,'fscore_STAR_train':A,'fscore_TOT_train':C,'fscore_STAR_test':B,
                                  'fscore_TOT_test':D,'S_STAR':S_STAR,'S_STAR_TOT':S_STAR_TOT},ignore_index=True) 
    

In [70]:
scores_STAR[:3]

,ModelName,Proportion,fscore_STAR_train,fscore_TOT_train,fscore_STAR_test,fscore_TOT_test,S_STAR,S_STAR_TOT
0,Naive Bayes,0.1,0.09,0.96,0.69,0.78,0.24,0.74
1,Naive Bayes,0.5,0.20,0.96,0.74,0.81,0.31,0.77
2,Naive Bayes,1.0,0.33,0.96,0.75,0.81,0.38,0.77


## Calculate New metric score for the GALAXY class and total score when GALAXY class is proportioned

In [71]:
GALAXY_train=Metrics_GALAXY[Metrics_GALAXY['Train/Test']=='Train']
GALAXY_test=Metrics_GALAXY[Metrics_GALAXY['Train/Test']=='Test']


scores_GALAXY=pd.DataFrame(columns=['ModelName','Proportion','fscore_GALAXY_train','fscore_TOT_train','fscore_GALAXY_test',
                                  'fscore_TOT_test','S_GALAXY','S_GALAXY_TOT'])

for model in model_names:
    prop=[0.1,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5,5.5,6,6.5,7,7.5,8,8.5,9,9.5,10]

    for i in prop:
        for index, row in GALAXY_train.iterrows():
        
            if (row['ModelName'] == model and row['Proportion']==i):
                A=row['fscore_GALAXY']
                C=row['fscore_TOT']
                
        for index, row in GALAXY_test.iterrows():
            if (row['ModelName'] == model and row['Proportion']==i):
                B=row['fscore_GALAXY']
                D=row['fscore_TOT']
            
        S_GALAXY=round((A+B)/(2*(1+abs(A-B))),2)
        S_GALAXY_TOT=round((C+D)/(2*(1+abs(C-D))),2)
        scores_GALAXY=scores_GALAXY.append({'ModelName':model,'Proportion':i,'fscore_GALAXY_train':A,'fscore_TOT_train':C,
                                        'fscore_GALAXY_test':B,'fscore_TOT_test':D,'S_GALAXY':S_GALAXY,'S_GALAXY_TOT':S_GALAXY_TOT},ignore_index=True) 


scores_GALAXY[:3]

,ModelName,Proportion,fscore_GALAXY_train,fscore_TOT_train,fscore_GALAXY_test,fscore_TOT_test,S_GALAXY,S_GALAXY_TOT
0,Naive Bayes,0.1,0.97,0.95,0.66,0.50,0.62,0.50
1,Naive Bayes,0.5,0.92,0.90,0.79,0.79,0.76,0.76
2,Naive Bayes,1.0,0.94,0.91,0.85,0.85,0.82,0.83


## Calculate New metric score for the QSO class and total score when QSO class is proportioned

In [72]:
QSO_train=Metrics_QSO[Metrics_QSO['Train/Test']=='Train']
QSO_test=Metrics_QSO[Metrics_QSO['Train/Test']=='Test']


scores_QSO=pd.DataFrame(columns=['ModelName','Proportion','fscore_QSO_train','fscore_TOT_train','fscore_QSO_test',
                                  'fscore_TOT_test','S_QSO','S_QSO_TOT'])

for model in model_names:
    prop=[0.1,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5,5.5,6,6.5,7,7.5,8,8.5,9,9.5,10]

    for i in prop:
        for index, row in QSO_train.iterrows():
        
            if (row['ModelName'] == model and row['Proportion']==i):
                A=row['fscore_QSO']
                C=row['fscore_TOT']
                
        for index, row in QSO_test.iterrows():
            if (row['ModelName'] == model and row['Proportion']==i):
                B=row['fscore_QSO']
                D=row['fscore_TOT']
            
        S_QSO=round((A+B)/(2*(1+abs(A-B))),2)
        S_QSO_TOT=round((C+D)/(2*(1+abs(C-D))),2)
        scores_QSO=scores_QSO.append({'ModelName':model,'Proportion':i,'fscore_QSO_train':A,'fscore_TOT_train':C,
                                        'fscore_QSO_test':B,'fscore_TOT_test':D,'S_QSO':S_QSO,'S_QSO_TOT':S_QSO_TOT},ignore_index=True) 


scores_QSO.head(3)

,ModelName,Proportion,fscore_QSO_train,fscore_TOT_train,fscore_QSO_test,fscore_TOT_test,S_QSO,S_QSO_TOT
0,Naive Bayes,0.1,0.85,0.87,0.85,0.87,0.85,0.87
1,Naive Bayes,0.5,0.84,0.87,0.84,0.86,0.84,0.86
2,Naive Bayes,1.0,0.84,0.87,0.83,0.86,0.83,0.86


In [82]:
QSO_test.to_csv('QSO_test.csv', index = False)
QSO_train.to_csv('QSO_train.csv',index = False)

## Combine the scores calculated for all the classes

In [73]:

df_scores = [scores_STAR, scores_GALAXY, scores_QSO]
scores = reduce(lambda left,right: pd.merge(left,right,on=['ModelName','Proportion']), df_scores)

In [74]:
scores.head(3)

,ModelName,Proportion,fscore_STAR_train,fscore_TOT_train_x,fscore_STAR_test,fscore_TOT_test_x,S_STAR,S_STAR_TOT,fscore_GALAXY_train,fscore_TOT_train_y,fscore_GALAXY_test,fscore_TOT_test_y,S_GALAXY,S_GALAXY_TOT,fscore_QSO_train,fscore_TOT_train,fscore_QSO_test,fscore_TOT_test,S_QSO,S_QSO_TOT
0,Naive Bayes,0.1,0.09,0.96,0.69,0.78,0.24,0.74,0.97,0.95,0.66,0.50,0.62,0.50,0.85,0.87,0.85,0.87,0.85,0.87
1,Naive Bayes,0.5,0.20,0.96,0.74,0.81,0.31,0.77,0.92,0.90,0.79,0.79,0.76,0.76,0.84,0.87,0.84,0.86,0.84,0.86
2,Naive Bayes,1.0,0.33,0.96,0.75,0.81,0.38,0.77,0.94,0.91,0.85,0.85,0.82,0.83,0.84,0.87,0.83,0.86,0.83,0.86


## Write scores to an ouput file

In [75]:
    scores.to_csv('scores.csv', index = False)

## Extract only the scores of all the target classes so that AUS(Area under the score can be calculated)

In [76]:
AUS_df=scores[['ModelName','Proportion','S_STAR','S_STAR_TOT','S_GALAXY','S_GALAXY_TOT','S_QSO','S_QSO_TOT']]

In [77]:
AUS_df.head(5)

,ModelName,Proportion,S_STAR,S_STAR_TOT,S_GALAXY,S_GALAXY_TOT,S_QSO,S_QSO_TOT
0,Naive Bayes,0.1,0.24,0.74,0.62,0.50,0.85,0.87
1,Naive Bayes,0.5,0.31,0.77,0.76,0.76,0.84,0.86
2,Naive Bayes,1.0,0.38,0.77,0.82,0.83,0.83,0.86
3,Naive Bayes,1.5,0.42,0.78,0.84,0.86,0.83,0.86
4,Naive Bayes,2.0,0.47,0.78,0.83,0.86,0.83,0.86


## Calculate AUS for all the scores

In [78]:
prop=[0.1,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5,5.5,6,6.5,7,7.5,8,8.5,9,9.5,10]
AUS=pd.DataFrame(columns=['ModelName','AUS_STAR','AUS_STAR_TOT','AUS_GALAXY','AUS_GALAXY_TOT','AUS_QSO','AUS_QSO_TOT'])


for model in model_names: 
    SUM_STAR=0
    SUM_STAR_TOT=0
    SUM_GALAXY=0
    SUM_GALAXY_TOT=0
    SUM_QSO=0
    SUM_QSO_TOT=0
    
    for index, row in AUS_df.iterrows():  
        
        if (row['ModelName'] == model and (row['Proportion']) == 0.1) or (row['ModelName'] == model and (row['Proportion'])==10):
            SUM_STAR = SUM_STAR+row['S_STAR']            
            SUM_STAR_TOT = SUM_STAR_TOT + row['S_STAR_TOT']
            SUM_GALAXY=SUM_GALAXY+row['S_GALAXY']
            SUM_GALAXY_TOT=SUM_GALAXY_TOT + row['S_GALAXY_TOT']
            SUM_QSO=SUM_QSO+row['S_QSO']
            SUM_QSO_TOT=SUM_QSO_TOT + row['S_QSO_TOT']
        elif(row['ModelName'] == model and (row['Proportion']) % 1== 0.5):
            SUM_STAR=SUM_STAR+(4*row['S_STAR'])
            SUM_STAR_TOT = SUM_STAR_TOT +(4* row['S_STAR_TOT'])
            SUM_GALAXY=SUM_GALAXY+(4*row['S_GALAXY'])
            SUM_GALAXY_TOT=SUM_GALAXY_TOT +(4*row['S_GALAXY_TOT'])
            SUM_QSO=SUM_QSO+(4*row['S_QSO'])
            SUM_QSO_TOT=SUM_QSO_TOT +(4*row['S_QSO_TOT'])
        elif(row['ModelName'] == model and (row['Proportion'])% 1.0== 0.0):
            SUM_STAR=SUM_STAR+(2*row['S_STAR'])
            SUM_STAR_TOT = SUM_STAR_TOT +(2* row['S_STAR_TOT'])
            SUM_GALAXY=SUM_GALAXY+(2*row['S_GALAXY'])
            SUM_GALAXY_TOT=SUM_GALAXY_TOT +(2*row['S_GALAXY_TOT'])
            SUM_QSO=SUM_QSO+(2*row['S_QSO'])
            SUM_QSO_TOT=SUM_QSO_TOT +(2*row['S_QSO_TOT'])
        

   
    AUS_STAR=round(((0.5/3)*SUM_STAR),2)
    AUS_STAR_TOT=round(((0.5/3)*SUM_STAR_TOT),2)
    AUS_GALAXY=round(((0.5/3)*SUM_GALAXY),2)
    AUS_GALAXY_TOT=round(((0.5/3)*SUM_GALAXY_TOT),2)
    AUS_QSO=round(((0.5/3)*SUM_QSO),2)
    AUS_QSO_TOT=round(((0.5/3)*SUM_QSO_TOT),2)
    AUS=AUS.append({'ModelName':model,'AUS_STAR':AUS_STAR,'AUS_STAR_TOT':AUS_STAR_TOT,'AUS_GALAXY':AUS_GALAXY,
                    'AUS_GALAXY_TOT':AUS_GALAXY_TOT,'AUS_QSO':AUS_QSO,'AUS_QSO_TOT':AUS_QSO_TOT},ignore_index=True) 
     

In [79]:
AUS

,ModelName,AUS_STAR,AUS_STAR_TOT,AUS_GALAXY,AUS_GALAXY_TOT,AUS_QSO,AUS_QSO_TOT
0,Naive Bayes,5.71,7.94,8.09,8.37,8.30,8.60
1,Logistic Regression,5.46,7.01,7.30,6.85,9.32,9.23
2,Decision Tree,9.62,9.51,9.87,8.88,10.00,9.83
3,KNN,6.65,6.79,6.57,4.37,7.96,7.33
4,Random Forest,9.13,9.26,9.79,9.01,9.97,9.81
5,AdaBoost,9.09,9.80,9.87,5.11,9.39,7.46
6,XGBoost,9.82,9.79,9.87,9.26,10.00,9.87
7,SVC,5.22,6.54,6.58,5.48,9.44,9.23


In [80]:
AUS.to_csv('AUS.csv', index = False)